# 7.2 케라스 콜백과 텐서보드를 사용한 딥러닝 모델 검사와 모니터링

### 7.2.1 콜백을 사용하여 모델의 훈련 과정 제어하기

모델을 훈련할 때 미리 예상할 수 없는 것들이 많습니다. 특히 최적의 검증 손실을 얻기 위해 얼마나 많은 에포크가 필요한지 알지 못합니다. 지금까지 예제는 적절한 훈련 에포크를 알아내기 위해 첫 번째 실행에서 과대적합이 시작될 때까지 충분한 에포크로 훈련했습니다. 그런 다음 최적의 에포크 횟수로 처음부터 새로운 훈련을 시작했습니다. 당연히 이런 방식은 낭비가 많습니다.

더 좋은 처리 방법은 검증 손실이 더 이상 향상되지 않을 때 훈련을 멈추는 것입니다. 케라스의 롤백을 사용하여 구현할 수 있습니다. 콜백은 모델의 상태와 성능에 대한 모든 정보에 접근하고 훈련 중지, 모델 저장, 가중치 적재 또는 모델 상태 변경 등을 처리할 수 있습니다.

콜백 사용 사례
- 모델 체크포인트 저장
- 조기 종료(early stopping)
- 훈련하는 동안 하이퍼파라미터 값을 동적으로 조정합니다
- 훈련과 검증 지표를 로그에 기록하거나 모델이 학습한 표현이 업데이터 될때마다 시각화합니다

keras.callbacks 모듈은 많은 내장 콜백을 포함하고 있습니다(전체 리스트 아님)
```python
keras.callbacks.ModelCheckpoint
keras.callbacks.EarlyStopping
keras.callbacks.LearningRateScheduler
keras.callbacks..ReduceLROnPlateau
keras.callbacks.CSVLogger
```

#### ModelCheckpoint와 EarlyStopping 콜백

EarlyStopping 콜백을 사용하면 정해진 에포크 동안 모니터링 지표가 향상되지 않을 때 훈련을 중지할 수 있습니다. 일반적으로 이 콜백은 훈련하는 동안 모델을 계속 저장해 주는 ModelCheckpoint와 함께 사용합니다.

```python
import keras

callbacks_list = [  # fit() 메서드의 callbacks 매개변수를 사용하여 콜백의 리스트를 모델로 전달합니다. 몇 개의 콜백이라도 전달할 수 있습니다.
    keras.callbacks.EarlyStopping(  # 성능 향상이 멈추면 훈련을 중지합니다.
        monitor='val_acc',  # 모델의 검증 정확도를 모니터링합니다.
        patience=1,  # 1 에포크보다 더 길게 (즉 2 에포크 동안) 정확도가 향상되지 않으면 훈련이 중지됩니다.
    ),
    keras.callbacks.ModelCheckpoint(  # 에포크마다 현재 가중치를 저장합니다.
        filepath='my_model.h5',  # 모델 파일의 경로
        # 이 두 매개변수는 val_loss가 좋아지지 낭흐면 모델 파일을 덮어쓰지 않는다는 뜻입니다. 훈련하는 동안 가장 좋은 모델이 저장됩니다.
        monitor='val_loss', 
        save_best_only=True,
    )
]

model.compie(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])  # 정확도를 모니터링하므로 모델 지표에 포함되어야 합니다.

# 콜백이 검증 손실과 검증 정확도를 모니터링하기 때문에 validation_data 매개변수에 검증 데이터를 전달해야 합니다.
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))
```

#### ReduceLROnPlateau 콜백

이 콜백을 사용하면 검증 손실이 향상되지 않을 때 학습률을 작게 할 수 있습니다.

```python
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',  # 모델의 검증 손실을 모니터링합니다.
        factor=0.1,  # 콜백이 호출될 때 학습률을 10배로 줄입니다.
        patience=10,  # 검증 손실이 10 에포크 동안 좋아지지 않으면 콜백이 호출됩니다.
    )
]

# 콜백이 검증 손실을 모니터링하기 때문에 validation_data 매개변수에 검증 데이터를 전달해야 합니다.
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))
```

#### 자신만의 콜백 만들기
콜백은 keras.callbacks.Callback 클래스를 상속받아 구현합니다.

```python
on_epoch_begin  # 각 에포크가 시작할 때 호출합니다.
on_epoch_end  # 각 에포크가 끝날 때 호출합니다.

on_batch_begin  # 각 배치 처리가 시작되기 전에 호출합니다.
on_batch_end  # 각 배치 처리가 끝난 후에 호출합니다.

on_train_begin  # 훈련이 시작될 때 호출합니다.
on_train_end  # 훈련이 끝날 때 호출합니다.
```

이 메서드들은 모두 logs 매개변수와 함께 호출됩니다. 이 매개변수에는 이전 배치, 에포크에 대한 훈련과 검증 측정값이 담겨 있는 딕셔너리가 전달됩니다. 또 콜백은 다음 속성을 참조할 수 있습니다.

- self.model: 콜백을 호출하는 모델 객체
- self.validaton_data: fit() 메서드에 전달된 검증 데이터

다음은 매 에포크의 끝에서 검증 세트의 첫 번째 샘플로 모델에 있는 모든 층의 활성화 출력을 계산하여 (넘파이 배열로) 디스크에 저장하는 자작 콜백의 예입니다.

```python
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    def set_model(self, model):  # 호출하는 모델에 대한 정보를 전달하기 위해 훈련하기 전에 호출됩니다.
        self.model = model
        layers_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)  # 각 층의 활성화 출력을 반환하는 Model 객체입니다.
        
    def on_epoch_end(self, spoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
            
        validation_sample = self.validation_data[0][0:1]  # 검증 데이터의 첫 번째 샘플을 가져옵니다.
        activations = self.activations_model.predict(validation_sample)
        # 배열을 디스크에 저장합니다.
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.savez(f, activations)
        f.close()
```

이것이 콜백에 대해 알아야 할 전부입니다.

### 7.2.2 텐서보드 소개: 텐서플로의 시각화 프레임워크

*tensorflow 1.6 부터 tensorboard 사용 가능*

좋은 연구를 하거나 좋은 모델을 개발하려면 실험하는 모델 내부에서 어떤 일이 일어나는지 자주 그리고 많은 피드백을 받아야 합니다.

텐서보드는 여러가지 멋진 기능을 제공합니다.
- 훈련하는 동안 측정 지표를 시각적으로 모니터링합니다.
- 모델 구조를 시각화합니다.
- 활성화 출력과 그래디언트의 히스토그램을 그립니다.
- 3D로 임베딩을 표현합니다.

#### 코드 7-7 텐서보드를 사용한 텍스트 분류 모델

In [2]:
# numpy 업데이트로 인한 추가
import numpy as np
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000  # 특성으로 사용할 단어의 수
max_len = 500  # 사용할 텍스트의 길이(가장 빈번한 max_features개의 단어만 사용합니다.)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

d:\git\jongkwangyun.github.io\aikerasstudy\aikeras\lib\site-packages\keras\datasets\imdb.py:99: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
d:\git\jongkwangyun.github.io\aikerasstudy\aikeras\lib\site-packages\keras\datasets\imdb.py:100: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


#### 코드 7-8 텐서보드 로그 파일을 위한 디렉터리 생성하기
```bash
$ mkdir my_log_dir
```

#### 코드 7-9 텐서보드 콜백과 함께 모델 훈련하기

In [3]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',  # 로그 파일이 기록될 위치입니다.
        histogram_freq=1,  # 1 에포크마다 활성화 출력의 히스토그램을 기록합니다.
        embeddings_freq=1,  # 1 에포크마다 임베딩 데이터를 기록합니다.
    )
]
history = model.fit(x_train, y_train,
                   epochs=20,
                   batch_size=128,
                   validation_split=0.2,
                   callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 5s 245us/step - loss: 0.6369 - acc: 0.6536 - val_loss: 0.4242 - val_acc: 0.8280
Epoch 2/20
20000/20000 [==============================] - 3s 141us/step - loss: 0.4426 - acc: 0.8171 - val_loss: 0.4289 - val_acc: 0.8298
Epoch 3/20
20000/20000 [==============================] - 3s 139us/step - loss: 0.3941 - acc: 0.7960 - val_loss: 0.4732 - val_acc: 0.7786
Epoch 4/20
20000/20000 [==============================] - 3s 141us/step - loss: 0.3385 - acc: 0.7679 - val_loss: 0.6236 - val_acc: 0.7292
Epoch 5/20
20000/20000 [==============================] - 3s 137us/step - loss: 0.2966 - acc: 0.7207 - val_loss: 0.5296 - val_acc: 0.6998
Epoch 6/20
20000/20000 [==============================] - 3s 142us/step - loss: 0.2494 - acc: 0.6769 - val_loss: 0.6458 - val_acc: 0.6196
Epoch 7/20
20000/20000 [==============================] - 3s 136us/step - loss: 0.2090 - acc: 0.6457 - val_loss: 0.6381 - v

```bash
$ tensorboard --logdir=my_log_dir
```

그다음 브라우저에서 http://localhost:6006 주소에 접속하면 모델의 훈련 결과를 확인할 수 있습니다.

모델의 층 그래프를 깔끔하게 그려주는 기능을 제공합니다.

```python
from keras.utils import plot_model

plot_model(model, to_file='model.png')
```

층 그래프에 크기 정보를 추가할 수 있습니다.

```python
from keras.utils import plot_model

plot_model(model, show_shapes=True, to_file='model.png')
```

### 7.2.3 정리

- 케라스 콜백은 훈련하는 동안 모델을 모니터링하고 모델 상태를 바탕으로 자동으로 작업을 수행하는 손쉬운 방법입니다.
- 텐서플로를 사용하면 텐서보드를 이용하여 모델 상황을 부라우저에서 시각화할 수 있습니다.